In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile,assemble
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Operator

Before any coding first lets define our binonmial options tree class which we will use in the future when we actually price our options contracts

In [2]:
#specific nodes of the binomial tree
class bnNode:
    def __init__(self, val):
        self.val = val #price of the stock at certain point at root will be strike price
        self.up = None #points to an upmovemnt node
        self.down = None #points to a downmovement node

    def __repr__(self):
        return f"bnNode(val={self.val})" #printer function to print curr val of the stock at given point

#the tree itself
class BinomialTree:
    def __init__(self, S0, upSz, downSz, NumStps):
        self.root = bnNode(S0) # root node to the strike price
        self.upSz = upSz #percent by which stock price increases ie increase of 10% in one step => 1.1
        self.downSz = downSz #percent by which stock price decreases ie decrease of 10% in one step => 0.9
        self.NumStps = NumStps #total number of steps in the tree itself
        self.build_tree(self.root, 0)

    def build_tree(self, node, step):
        if step < self.NumStps:
            node.up = bnNode(node.val * self.upSz)
            node.down = bnNode(node.val * self.downSz)

            #recrusively build the rest of the tree
            self.build_tree(node.up, step + 1)
            self.build_tree(node.down, step + 1)




In [3]:

#simply defines an up-movement in stock price and by how much
def up_move(sigma, t):
    b = np.exp(sigma * np.sqrt(1/t)) - 1
    return b

#this consctructs the quantum operator whcih is a matrix
#that represents the possible price movements of the stock
#diagonal matrix
def build_quantum_op(b, r, t):
    a = -b  # Symmetric up and down movements
    A = np.array([[1+b, 0], [0, 1+a]])
    return A

# Parameters in math notation according to the paper
sigma = 0.1 #volatility
r = 0.05 #risk free interest rate
T = 1 # total time period
N = 3 #number of steps in the binomial model
t = T / N # time increment per step

b = up_move(sigma, t) #up movement factor
A = build_quantum_op(b, r, t)
eigenvalues, eigenvectors = np.linalg.eigh(A)




The initialization of the density matrix for the quantum binomial model is given by the equation:

\[
\rho = \bigotimes_{j=1}^{N} \left( |u_j\rangle \langle u_j| (1 - q) + |v_j\rangle \langle v_j| q \right)
\]

where:
- \( \bigotimes \) denotes the tensor product,
- \( N \) is the number of steps,
- \( |u_j\rangle \) and \( |v_j\rangle \) represent the quantum states for up and down movements, respectively,
- \( 1 - q \) is the probability of an up movement,
- \( q \) is the probability of a down movement.




In [5]:
# Now we need to intialize density matrix using the
# eigenvectors and constructing mixed stat for each qubit


#pass in a ref to our binomial options tree root node
def buildDensityMatr(BinTree:BinomialTree):
  intial_state = np.zeros(2**BinTree.NumStps,dtype=complex) #total number of states N is the number of steps in the model as such need 2^N states as each step has an up step and a down step



#Calculate q or the risk-neutral probability whcih we will represent by
#the variable risk_neut_prob
  risk_neut_prob = (np.exp(r*t) - BinTree.downSz)/(BinTree.upSz-BinTree.downSz)

  #recusively build the density matrix starting at the root node of the bintree and going throuhg the entire tree
  #updating the matrix each way as it goes through each step
  def recursive_build(node):
      #ie no other nodes left so we make the matrix itself here 
      if node.up is None and node.down is None:
            # np.outer it a function that does an outer-product it will reutrn a matrix 
            # this is going to construct the density matrix from the state with the first column of the eigenvector and its complex conjugate
            # one example of an outer product and what it will look like is if we have: The outer product of [1, 0] and [1, 0] is: [[1*1, 1*0],[0*1,0*0]] resulting in [[1, 0],[0,0]]
            #another representation is first row: [1*1, 1*0] second row: [0*1,0*0] resulting in first row [1,0] second row [0,0]
          return np.outer(eigenvectors[:, 0], np.conj(eigenvectors[:, 0])) #
      else:
          # Recursively build density matrices for child nodes
        
          up_density = recursive_build(node.up)
          down_density = recursive_build(node.down)
          # we finally get to the end of the tree when we get to this line.  Then we can calculat the node density below and update the density matrix
          # Then the up density and down Density will be a matrix itself that is going to be updated using the previous densities so 
          
          # Calculate node density as weighted sum of up and down densities

            #the best way to think of this is that when we get to the end of the tree ie last node or leaf node this leaf node points to nothing and is the end
            # based on our binomial tree each node will either have two children or no children like a leaf node so this leaf node is either a up movement or a down movement
            # visual:


              #          __node1_up -> null
              #        _/
              #-node 1  
              #       _
              #        \
              #         ⎻⎻ node1_down ->null
                        
                  # the node1_up val will be unique from the node1 down becaus w are using the eigenvctors to represent different 
                  #quantum states and so each leaf node corresponds to a unique state derived from the binomial tree's possible outcomes


            # Example: if up_density = [[1, 0], [0, 0]] and down_density = [[0, 0], [0, 1]], with risk_neut_prob = 0.6
            # node_density = 0.4 * [[1, 0], [0, 0]] + 0.6 * [[0, 0], [0, 1]]
            #              = [[0.4, 0], [0, 0]] + [[0, 0], [0, 0.6]]
            #              = [[0.4, 0], [0, 0.6]]

            
          node_density = (1 - risk_neut_prob) * up_density + risk_neut_prob * down_density
          return node_density

    # Start recursion from the root node
  root_density = recursive_build(BinTree.root)
  return root_density

##
The below code is still to be completed the above code should work
##

In [22]:
qc = QuantumCircuit(N)

# Assuming the same dynamics on each qubit
for i in range(N):
    initial_state = eigenvectors[:, 0]  # Use the first eigenvector
    qc.initialize(initial_state, [i])  # Initialize each qubit independently


In [23]:
def apply_stock_movements(qc, num_periods, up_theta, down_theta):
    for i in range(num_periods):
        qc.h(i)  # Create superposition
        qc.cry(up_theta, i, (i + 1) % num_periods)  # Controlled-RY for up movement
        qc.cry(down_theta, i, (i + 1) % num_periods)  # Controlled-RY for down movement

apply_stock_movements(qc, N, np.pi/4, -np.pi/4)
qc.measure_all()


In [25]:
simulator = AerSimulator()
transpiled_circuit = transpile(qc, simulator)
qobj = assemble(transpiled_circuit, shots=1000)
result = simulator.run(qobj).result()
counts = result.get_counts(qc)
print(counts)


{'100': 113, '110': 132, '011': 145, '000': 122, '001': 142, '010': 119, '101': 118, '111': 109}


C:\Users\chaha\AppData\Local\Temp\ipykernel_9516\2237938607.py:4: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.14 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = simulator.run(qobj).result()


In [26]:
#set strike price
K=50
def calculate_payoffs(final,K):
    payoffs = {k: max(int(k) - K, 0) for k in final_prices.keys()}
    return payoffs

# Assuming 'counts' is a dictionary where keys are final stock prices and values are their frequencies
final_prices = {'101': 500, '110': 300, '111': 200}  # Example output of quantum simulation
payoffs = calculate_payoffs(final_prices, K)
print(payoffs)

{'101': 51, '110': 60, '111': 61}
